In [ ]:
import h2o
from h2o.automl import H2OAutoML
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-mar-2021/train.csv")
train

In [ ]:
train.target.value_counts().to_frame()

In [ ]:
h2o.init(max_mem_size='12G', nthreads=4)

In [ ]:
train = h2o.import_file("../input/tabular-playground-series-mar-2021/train.csv")
test = h2o.import_file("../input/tabular-playground-series-mar-2021/test.csv")

x = train.columns
y = "target"
#x = x.drop("target").column
#x.remove(y)

# Identify predictors and response
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()


#train = h2o.H2OFrame(train)

In [ ]:
object_columns = x[:19]
for f in object_columns:
    train[f] = train[f].asfactor()
    test[f] = test[f].asfactor()

In [ ]:
train

In [ ]:
test

In [ ]:
'''
# create a fold column with 5 folds
# randomly assign fold numbers 0 through 4 for each row in the column
fold_numbers = train.kfold_column(n_folds = 5, seed = 1234)

# rename the column "fold_numbers"
fold_numbers.set_names(["fold_numbers"])


# append the fold_numbers column to the cars dataset
train = train.cbind(fold_numbers)# append the fold_numbers column to the cars dataset
train = train.cbind(fold_numbers)
'''
# set the number of folds for you n-fold cross validation:
folds = 10
# folds = 10

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1234, max_runtime_secs=10800, 
                 stopping_metric = "auc", nfolds = folds, 
                sort_metric = "auc", exclude_algos = ["DeepLearning"]) 
aml.train(x=x, y=y, training_frame=train) # , fold_column="fold_numbers"

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
board = aml.leader
board

In [ ]:
preds = aml.predict(test).as_data_frame()
preds.head()

In [ ]:
sub_df = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
sub_df['target'] = preds['predict']
sub_df.to_csv('submission_h2o.csv', index=False)
sub_df.head()

sub_automl = preds['predict']

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")

In [ ]:
sub.target.value_counts().to_frame()

References:

https://www.kaggle.com/asimzahid/h2o-tps-feb-2021-automl

# upvote if find this interesting
